<a href="https://colab.research.google.com/github/lokeshkumarn/Notebooks/blob/master/DeepSpeechv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !wget -O git-lfs.tar.gz https://github.com/git-lfs/git-lfs/releases/download/v2.4.2/git-lfs-linux-amd64-2.4.2.tar.gz
# !tar -xvzf git-lfs.tar.gz
# !./git-lfs-2.4.2/install.sh
# %cd "/content/drive/My Drive/"
# !pwd
!git clone https://github.com/mozilla/DeepSpeech
%cd "/content/DeepSpeech"
!pwd
!pip install -r requirements.txt
!pip uninstall -y tensorflow
!pip uninstall -y tensorflow-gpu
!pip install deepspeech-gpu
!pip install tensorflow-gpu==1.11.0

In [1]:
%cd "/content/DeepSpeech"

/content/DeepSpeech


**Fetching the pre-trained model & sample audio clip**

In [0]:
#!wget -O - https://github.com/mozilla/DeepSpeech/releases/download/v0.1.1/deepspeech-0.1.1-models.tar.gz | tar xvfz -
#!wget -O - https://github.com/mozilla/DeepSpeech/releases/download/v0.5.1/deepspeech-0.5.1-models.tar.gz | tar xvfz -
!wget -O - https://github.com/mozilla/DeepSpeech/releases/download/v0.6.0/deepspeech-0.6.0-models.tar.gz | tar xvfz -
#!wget -O - https://github.com/mozilla/DeepSpeech/releases/download/v0.1.1/deepspeech-0.1.1-checkpoint.tar.gz | tar xvfz -
#!wget -O - https://github.com/mozilla/DeepSpeech/releases/download/v0.5.1/deepspeech-0.5.1-checkpoint.tar.gz | tar xvfz -
!wget -O - https://github.com/mozilla/DeepSpeech/releases/download/v0.6.0/deepspeech-0.6.0-checkpoint.tar.gz | tar xvfz -
!mkdir customdata                              
!wget -O customdata/sample-1.wav https://github.com/mayukhnair/dsdatasets/raw/master/sample-1.wav
!wget -O customdata/ttssample1.wav https://github.com/mayukhnair/dsdatasets/raw/master/ttssample1.wav

**GPU Training Setup**

In [0]:
#!python "/content/drive/My Drive/DeepSpeech/util/taskcluster.py" --target "/content/drive/My Drive/DeepSpeech/native_client_new"
#!apt-get install dirmngr
#!wget -O cuda9base.deb https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb

In [0]:
!dpkg -i cuda9base.deb

Selecting previously unselected package cuda-repo-ubuntu1704-9-0-local.
(Reading database ... 145674 files and directories currently installed.)
Preparing to unpack cuda9base.deb ...
Unpacking cuda-repo-ubuntu1704-9-0-local (9.0.176-1) ...
Setting up cuda-repo-ubuntu1704-9-0-local (9.0.176-1) ...


In [0]:
!ls -l

In [0]:
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub

OK


In [0]:
!apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1704/x86_64/7fa2af80.pub

Executing: /tmp/apt-key-gpghome.6Zev4M1KL0/gpg.1.sh --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1704/x86_64/7fa2af80.pub
gpg: requesting key from 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1704/x86_64/7fa2af80.pub'
gpg: key F60F4B3D7FA2AF80: "cudatools <cudatools@nvidia.com>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1


In [0]:
!apt-get update

In [0]:
!apt-get install cuda=9.0.176-1

In [0]:
!export LD_LIBRARY_PATH=/usr/local/cuda-9.0/lib64:$LD_LIBRARY_PATH

In [0]:
!apt install nvidia-cuda-toolkit

In [0]:
!wget -O cudnn7.tgz https://www.dropbox.com/s/dgmc6extwj8u14m/cudnn-9.0-linux-x64-v7.tgz

In [0]:
!tar -xvzf cudnn7.tgz

cuda/include/cudnn.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
tar: cuda/lib64/libcudnn.so: Cannot create symlink to ‘libcudnn.so.7’: Operation not supported
cuda/lib64/libcudnn.so.7
tar: cuda/lib64/libcudnn.so.7: Cannot create symlink to ‘libcudnn.so.7.0.5’: Operation not supported
cuda/lib64/libcudnn.so.7.0.5
cuda/lib64/libcudnn_static.a
tar: Exiting with failure status due to previous errors


In [0]:
!cp -P cuda/include/cudnn.h /usr/include

In [0]:
!cp -P cuda/lib64/libcudnn* /usr/lib/x86_64-linux-gnu/

In [0]:
!chmod a+r /usr/lib/x86_64-linux-gnu/libcudnn*
!cp cuda/include/cudnn.h /usr/local/cuda/include
!cp cuda/lib64/libcudnn* /usr/local/cuda/lib64
!chmod a+r /usr/local/cuda/lib64/libcudnn*

**Deep Speech Model Load, STT**

In [0]:
import deepspeech
from deepspeech import *
from deepspeech import Model
import scipy.io.wavfile as wav

import glob
import webrtcvad
import logging
# import wavSplit
from deepspeech import Model
from timeit import default_timer as timer

In [0]:
def resolve_models(dirName):
    pb = glob.glob(dirName + "/*.pb")[0]
    logging.debug("Found Model: %s" % pb)

    lm = glob.glob(dirName + "/lm.binary")[0]
    trie = glob.glob(dirName + "/trie")[0]
    logging.debug("Found Language Model: %s" % lm)
    logging.debug("Found Trie: %s" % trie)

    return pb, lm, trie
    
def load_model(models, lm, trie):
    BEAM_WIDTH = 500
    LM_ALPHA = 0.75
    LM_BETA = 1.85

    model_load_start = timer()
    ds = Model(models, BEAM_WIDTH)
    model_load_end = timer() - model_load_start
    logging.debug("Loaded model in %0.3fs." % (model_load_end))

    lm_load_start = timer()
    ds.enableDecoderWithLM(lm, trie, LM_ALPHA, LM_BETA)
    lm_load_end = timer() - lm_load_start
    logging.debug('Loaded language model in %0.3fs.' % (lm_load_end))

    return [ds, model_load_end, lm_load_end]

def stt(ds, audio, fs):
    inference_time = 0.0
    audio_length = len(audio) * (1 / fs)

    # Run Deepspeech
    logging.debug('Running inference...')
    inference_start = timer()
    output = ds.stt(audio)
    inference_end = timer() - inference_start
    inference_time += inference_end
    logging.debug('Inference took %0.3fs for %0.3fs audio file.' % (inference_end, audio_length))

    return [output, inference_time]

In [5]:
output_graph, lm, trie = resolve_models('deepspeech-0.6.0-models')
output_graph, lm, trie

('deepspeech-0.6.0-models/output_graph.pb',
 'deepspeech-0.6.0-models/lm.binary',
 'deepspeech-0.6.0-models/trie')

In [6]:
model_retval = load_model(output_graph, lm, trie)
model_retval 

 87.53640789899998,
 0.0003377859999886823]

In [7]:
fs, audio = wav.read("customdata/ttssample1.wav")
output = stt(model_retval[0], audio,fs)
print(output)

['who unshaven the the', 2.8817622469999833]
